In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import my_txtutils

In [2]:
mypath='./new_docs/'
txts='./txts/'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['acts_new.txt', 'gal_eph_new.txt', 'heb_new.txt', 'jam_jud_new.txt', 'john_new.txt', 'jud_rev_new.txt', 'luke_8_john_new.txt', 'mark01_new.txt', 'matt02_new.txt', 'matt_new.txt', 'phil_col_new.txt', 'rom_new.txt', 'thes_tim_new.txt', 'tit_phl_new.txt']


In [261]:
#large files
def large_f():
    filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

In [ ]:
#small files
def small_f():
    filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                outfile.write(infile.read())

In [ ]:
#leaking descriptor
def leak_df():
    filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for line in itertools.chain.from_iterable(itertools.imap(open, filnames)):
            outfile.write(line)

In [8]:
!h:/setpath


(base) H:\Downloads\src\notebooks\research\ok-rnn>set path=C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Library\bin;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Library\bin;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Scripts;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Library\bin;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Library\mingw-w64\bin;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Library\usr\bin;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Library\bin;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\Scripts;C:\Users\u1273400\AppData\Local\Continuum\Anaconda3\bin;C:\SIMULIA\Commands;C:\Program Files\Microsoft MPI\Bin\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\ProgramData\Oracle\Java\javapath;C:\Program Files (x86)\ActiveState Komodo Edit 10\;C:\windows\system32;C:\windows;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0\;C:\PROGRAM FILES

The system cannot find the path specified.
The system cannot find the path specified.


In [10]:
!lmplz -o 5 -S 50 -T C:/Users/u1273400/itmp < H:/Downloads/src/notebooks/research/ok-rnn/new_txts/ok1906180830c.txt > H:/Downloads/src/notebooks/research/ok-rnn/new_txts/ok.arpa

'lmplz' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(ok02+'.meta')
    new_saver.restore(sess, ok02)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    for i in range(100):
        yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})

        # If sampling is be done from the topn most likely characters, the generated text
        # is more credible and more "english". If topn is not set, it defaults to the full
        # distribution (ALPHASIZE)

        # Recommended: topn = 10 for intermediate checkpoints, topn=2 or 3 for fully trained checkpoints

        c = my_txtutils.sample_from_probabilities(yo, topn=2)
        y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(my_txtutils.convert_to_alphabet(c))
        print(c, end="")

        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            ncnt = 0

In [3]:
t={}
for f in onlyfiles:
    inF=open(mypath+f,"r")
    t[f]=inF.read()


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2952: character maps to <undefined>

Let's remove punctuation symbols
Punctuation Rules in English
1. the period (or full stop in British English)
2. the comma.
3. the exclamation mark.
4. the question mark.
5. the colon.
6. the semicolon.
7. the quotation mark.
8. the apostrophe.

In [12]:
def stripper(src):
    return src.lower().replace("\n"," ").replace("  "," ").replace("ẹ","e")\
    .replace("!","").replace("?","").replace("“","").replace("ḍ","d")\
    .replace(":","").replace(";","").replace("ị","i").replace("ḅ","b")\
    .replace("'","").replace("”","").replace("ọ","o").replace("ú","u")\
    .replace("'","").replace("”","").replace("ụ","u").replace('"',"")

In [ ]:
for f in t:
    t[f]=stripper(t[f])
    oF=open(txts+f,"a")
    of.write(t[f])